## The Dataset

We implement the code by dataset `ballroom` [1], which has 8 genres(`Cha Cha`, `Jive`, `Quickstep`, `Rumba`, `Samba`, `Tango`, `Viennese Waltz`, and `Slow Waltz`). Every audio is 30s fixed length.

[1] [ballroom](http://mtg.upf.edu/ismir2004/contest/tempoContest/node5.html)


In [10]:
# Import necessary lib
import librosa
from pathlib import Path
import soundfile as sf


## List of music type, tempos, and keys

In [11]:
# Tempo list
tmpo_fctr = {"original": 1.00,
             "x125": 1.25,
             "x150": 1.50,
             "x075": 0.75,
             "x050": 0.50}
# Key
keys = [-2, -1, 0, 1, 2]

# Music Type
music_type = ['ChaChaCha',
              'Jive',
              'Quickstep',
              'Rumba-American',
              'Rumba-International',
              'Rumba-Misc',
              'Samba',
              'Tango',
              'VienneseWaltz',
              'Waltz']


## Time Stretch

This function implements tempo modification.


In [12]:
def time_stretch(signal, time_stretch_rate):
    return librosa.effects.time_stretch(y=signal, rate=time_stretch_rate)


## Pitch Scaleing

This function implements key modification.

In [13]:
def pitch_scale(signal, sr, num_semitones):
    return librosa.effects.pitch_shift(y=signal, sr=sr, n_steps=num_semitones)


## Get Key (Not yet finished)
This function gets tonic and mode of audio>

In [14]:
import numpy as np

# Not yet finish.
def get_music_key(y, sr):
    # pitches in 12 tone equal temperament
    pitches = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    song_chroma = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr), axis=1)
    key = dict()

    # select the most dominate pitch
    pitch_id = np.argmax(song_chroma)
    key['pitch'] = pitches[pitch_id]

    min_third_id = (pitch_id+3) % 12
    maj_third_id = (pitch_id+4) % 12

    # check if the musical 3rd is major or minor
    if song_chroma[min_third_id] < song_chroma[maj_third_id]:
        key['third'] = 'Major'
    elif song_chroma[min_third_id] > song_chroma[maj_third_id]:
        key['third'] = 'Minor'

    return key


## Synthesis audio bank by modify tempo and key

We synthesis audiobank by modifying 5 different tempos, and 5 different keys. Every original audio file has 25 different combination of modification. We pick 2 audios randomly from each genres, noted that datadet `ballroom` seperate genre `Rumba` into subset American, International and Misc. So we will have 20 original audio files(pick 2 sudio files randomly 10 genres), each original audio has 25 different combination of modification, which is 500 synthesis audiobanks in the end.

In [15]:
import random
import pandas as pd
# from xai_music import audio


_WAV_PATH = rf'audio/wav'
_AUGMENT_PATH = rf'audio/augments'
_WAV = '.wav'

wav_audio_lst = list()


# # Load list of audios from dataset
# with open(audiobank.config.AUDIO_FILE_PATH, 'r') as f:
#     music_path = f.read().splitlines()
# music_path = [m.split('/') for m in music_path]
# df_music_types = pd.DataFrame(music_path, columns=['', 'type', 'music'])
# df_music_types.drop(df_music_types.columns[[0]], axis=1, inplace=True)




In [16]:
import random
import pandas as pd
from tqdm import tqdm

_WAV_PATH = rf'audio/wav'
_AUGMENT_PATH = rf'audio/augments'
_WAV = '.wav'

wav_audio_lst = list()


# Load list of audios from dataset
with open('audio/wav/allBallroomFiles', 'r') as f:
    music_path = f.read().splitlines()
music_path = [m.split('/') for m in music_path]
df_music_types = pd.DataFrame(music_path, columns=['', 'type', 'music'])
df_music_types.drop(df_music_types.columns[[0]], axis=1, inplace=True)

# Pick 2 original audio file randomly from list
for mt in music_type:
    tm_lst = df_music_types.loc[df_music_types['type'] == mt].sample(n=2)
    for row in tm_lst.itertuples(index=False):
        wav_audio_lst.append(rf"{_WAV_PATH}/{row.type}/{row.music}")

for wa in tqdm(wav_audio_lst, desc="Processing WAV files"):
    # Load the audio as a waveform `y`
    # Store the sampling rate as `sr`
    y, sr = librosa.load(rf'{wa}')

    # With diferent tempo
    # scale (in this case stretch) the overall tempo by this factor
    for tmp_n, tmp_v in tmpo_fctr.items():
        augments_1_audio = time_stretch(y, tmp_v)

        # With diferent key
        for key in keys:
            augments_2_audio = pitch_scale(augments_1_audio, sr, key)
            # key_analyze = get_music_key(augments_2_audio, sr)
            key_note = ''
            file_name = ''
            if key == 0:
                key_note = 'original'
            else:
                key_note = f"{key}"
            
            m_tyep=str(wa).split('/')[-2]
            file_name = f"{m_tyep}-{str(wa).split('/')[-1][:-4]}|{tmp_n}|{key_note}{_WAV}"
            sf.write(Path(_AUGMENT_PATH, file_name), augments_2_audio, sr)


Processing WAV files:   0%|          | 0/20 [00:00<?, ?it/s]

Processing WAV files: 100%|██████████| 20/20 [01:50<00:00,  5.50s/it]


## Load a trained neural network model


In [17]:
import tensorflow as tf
from musicnn_keras.tagger import top_tags


musicnn = tf.keras.models.load_model(
    './musicnn_keras/keras_checkpoints/MSD_musicnn.h5')


2024-04-13 14:16:39.683588: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 14:16:39.685525: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 14:16:39.712245: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 14:16:39.713357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 14:16:40.596028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Get taggs

In [18]:
import json
from tqdm import tqdm

_MP3_PATH = rf'audio/augments'

audios = Path(_MP3_PATH).glob('*.wav')
audio_json = dict()

for af in tqdm(audios):
    # Get Tags
    audio_dict = dict()
    audio_dict['tags'] = top_tags(str(af),
                                  model='MTT_musicnn',
                                  topN=10,
                                  print_tags=False)

    # Save result
    audio_name = str(af).split('/')[-1]
    meta_data = audio_name.split('.')[0].split('|')
    audio_dict['music'] = meta_data[0]
    audio_dict['tempo'] = meta_data[1]
    audio_dict['key'] = meta_data[2]
    audio_json[audio_name.split('.')[0]] = audio_dict

with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(audio_json, f, ensure_ascii=False, indent=4, sort_keys=True)


0it [00:00, ?it/s]

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

1it [00:00,  1.26it/s]

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

2it [00:01,  1.73it/s]

...

923it [07:45,  1.85it/s]

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

924it [07:45,  1.81it/s]

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

925it [07:46,  1.98it/s]
